# Fine tunning pre-trained Yolo model on fire detection use case
### Data was loaded on google drive
### Model is logged to MLFLOW server on AWS (not runnning anymore, we use AzureML instead)

In [ ]:
!pip install ultralytics torch mlflow boto3

In [ ]:
from ultralytics import YOLO

In [ ]:
!yolo task= detect mode = train model = yolov8n.pt data = /content/drive/MyDrive/fire_data/data.yaml epochs = 15 imgsz = 640

100% 6.25M/6.25M [00:00<00:00, 205MB/s]
Ultralytics YOLOv8.2.64 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/fire_data/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

# Log model to MLFlow artifact

In [ ]:
import mlflow
import subprocess
import os

In [ ]:
MLFLOW_TRACKING_URI = "https://mlflow-server.duckdns.org/"
# Set the tracking URI for MLflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# MLflow will create an experiment if it doesn't exist
mlflow.set_experiment("fire_detection_yolo")

2024/07/24 21:38:23 INFO mlflow.tracking.fluent: Experiment with name 'fire_detection_yolo' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/251345887274286367', creation_time=1721857103416, experiment_id='251345887274286367', last_update_time=1721857103416, lifecycle_stage='active', name='fire_detection_yolo', tags={}>

In [ ]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 's3://mlflow-bucket42'

In [ ]:
model_path = "/content/fire_yolo.pt"
model = YOLO(model_path)

with mlflow.start_run() as run:
    # Log the YOLO model using mlflow.pytorch
    mlflow.pytorch.log_model(model, "model")
    mlflow.log_artifact(model_path, "model")
    mlflow.log_param("epochs", 15)
    mlflow.log_metric("accuracy", 0.93)


2024/07/24 23:18:14 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/07/24 23:18:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp3v5gbzyk/model/data, flavor: pytorch). Fall back to return ['torch==2.3.1', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
